In [1]:
from htmd import *
htmd.config(viewer='ngl')


Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049

You are on the latest HTMD version (unpackaged : /home/ramon/progs/htmd/htmd).



In [2]:
ternary = Molecule("./ternary_endblocked-corr.pdb")
tern_op, prepData = proteinPrepare(ternary, returnDetails=True)
prepData

2017-02-01 19:19:42,093 - propka - INFO - No pdbfile provided
2017-02-01 19:20:51,823 - htmd.builder.preparationdata - INFO - The following residues are in a non-standard state: HIS    44  A (HID), HIS   153  A (HIE), HIS   246  A (HIE), HIS   299  B (HID), CYS   390  B (CYX), HIS   408  B (HIE), CYS   417  B (CYX), HIS   501  B (HIE), HIS   554  C (HID), HIS   663  C (HIP), HIS   756  C (HIE), DA      1  F (DA5), DG     40  F (DG3), DC      1  G (DC5), DT     40  G (DT3)
2017-02-01 19:20:51,828 - htmd.builder.preparationdata - WARNING - Dubious protonation state: the pKa of 6 residues is within 1.0 units of pH 7.0.
2017-02-01 19:20:51,829 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS    44  A (pKa= 6.30)
2017-02-01 19:20:51,830 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS   299  B (pKa= 6.25)
2017-02-01 19:20:51,831 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS   554  C (pKa= 6.49)
2017-02-01

PreparationData object about 985 residues.
Please find the full info in the .data property, e.g.: 
  resname  resid insertion chain        pKa protonation flipped     buried
0     MET      1               A        NaN         MET     NaN        NaN
1     PHE      2               A        NaN         PHE     NaN        NaN
2     GLU      3               A   3.639572         GLU     NaN   0.000000
3     ALA      4               A        NaN         ALA     NaN        NaN
4     ARG      5               A  14.045757         ARG     NaN  14.285714
 . . .

In [3]:
tern_op.set('segid', 'A', sel='chain A')
tern_op.set('segid', 'B', sel='chain B')
tern_op.set('segid', 'C', sel='chain C')
tern_op.set('segid', 'D', sel='chain D')
tern_op.set('segid', 'E', sel='chain E')
tern_op.set('segid', 'F', sel='chain F')
tern_op.set('segid', 'G', sel='chain G')

In [4]:
tern_op.center()

In [5]:
from htmd.molecule.util import maxDistance
D = maxDistance(tern_op, 'all')
print(D)

70.9604328331


In [6]:
smol = solvate(tern_op, pad=5., rotate=False) #Rotation not implemented yet

2017-02-01 19:20:53,647 - htmd.builder.solvate - INFO - Using water pdb file at: /home/ramon/progs/htmd/htmd/builder/wat.pdb
2017-02-01 19:20:54,701 - htmd.builder.solvate - INFO - Replicating 12 water segments, 2 by 3 by 2


Solvating: 100% (12/12) [##########################################] eta 00:00 /


2017-02-01 19:21:35,696 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 41780 water molecules were added to the system.


In [7]:
#bmol = amber.build(smol, outdir='./build')
caps = {}
caps['A'] = ['none', 'none']
caps['B'] = ['none', 'none']
caps['C'] = ['none', 'none']

caps['D'] = ['ACE', 'NME']  # Enables capping for segment A
caps['E'] = ['ACE', 'NME']  # Enables capping for segment A

ffs = ['leaprc.ff14SB',]
param = ['frcmod.ionsjc_tip3p','/home/ramon/calculs/PCNA/HTMD/params/parmBSC1.frcmod']
atomtypes=  (( "C1", "C", "sp2" ),
  ( "C2", "C", "sp2" ),
  ( "CI", "C", "sp3" ),
  ( "CE", "C", "sp3" ))
ss_bonds = [
    DisulfideBridge('A', 135, 'A', 162),
    DisulfideBridge('B', 390, 'B', 417),
    DisulfideBridge('C', 645, 'C', 672),
]

#ss_bonds =np.vstack((smol.atomselect("resid 135 162 and name SG", indexes=True),
#smol.atomselect("resid 390 417 and name SG", indexes=True),
#smol.atomselect("resid 645 672 and name SG", indexes=True),))

#molbuilt = amber.build(mol, ff=ffs, caps=caps, outdir='/tmp/build', saltconc=0.15)
bmol = amber.build(smol, atomtypes=atomtypes, param = param, disulfide=ss_bonds,
                   offlibraries='./params/parmBSC1.lib', outdir='./build/',
                  ff=ffs, caps=caps, saltconc=0.1)


2017-02-01 19:21:40,617 - htmd.builder.amber - INFO - Converting CHARMM membranes to AMBER.
2017-02-01 19:22:41,756 - htmd.builder.amber - INFO - Path frcmod.ionsjc_tip3p not found, assuming a standard AmberTools file.
2017-02-01 19:22:41,757 - htmd.builder.amber - INFO - Writing PDB file for input to tleap.
2017-02-01 19:22:43,300 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2017-02-01 19:22:54,522 - htmd.builder.amber - INFO - Starting the build.
2017-02-01 19:23:50,160 - htmd.builder.amber - INFO - Finished building.
2017-02-01 19:23:56,365 - htmd.builder.ionize - INFO - Adding 0 anions + 110 cations for neutralizing and 156 ions for the given salt concentration.
2017-02-01 19:23:57,529 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2017-02-01 19:23:57,530 - htmd.builder.ionize - INFO - Min distance between ions: 5A
2017-02-01 19:23:57,530 - htmd.builder.ionize - INFO - Placing 266 ions.
2017

In [12]:
smol.atomselect('nucleic and not name P').sum()

2460